In [1]:
import torch
import numpy as np
import random 
import matplotlib.pyplot as plt

# Check that MPS is available
if torch.backends.mps.is_available():
    DEVICE = torch.device("cpu")
elif torch.cuda.is_available():
    DEVICE = torch.device("mps")
else:
    DEVICE = torch.device("cpu")
print(DEVICE)

cpu


In [2]:

""" Example of graph classification problem """
from tqdm import tqdm
from torch_geometric.nn import global_mean_pool as gap
from torch_geometric.nn import GCNConv, SAGEConv
from torch.nn import Linear, ModuleList, ReLU, Tanh, Sequential
import torch.nn.functional as F
import os.path as osp
import sys
sys.path.append('/Users/krishnanraghavan/Documents/Projects/Poseidon/graph_nn_2')


import random
from datetime import datetime
from psd_gnn.dataset import Merge_PSD_Dataset, PSD_Dataset
# from psd_gnn.models.graph_classifier import GNN
from psd_gnn.utils import process_args
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score)
from sklearn.model_selection import train_test_split
from torch.nn import CrossEntropyLoss
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.loader import DataLoader, ImbalancedSampler, NeighborLoader
from torch.distributions import LogNormal
from torch.optim.lr_scheduler import ReduceLROnPlateau
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)

The model definition

In [3]:

class GNN(torch.nn.Module):
    def __init__(self,
                 n_node_features: int,
                 n_hidden: int,
                 n_output: int,
                 n_conv_blocks: int = 1 ) -> None:
        """ Init the GNN model (new version).

        Args:
            n_node_features (int): Number of features at node level.
            n_edge_features (int): Number of features at edge level.
            n_hidden (int): Number of hidden dimension.
            n_output (int): number of output dimension
            n_conv_blocks (int): Number of
        """
        # super class the class structure
        super().__init__()

        # add the ability to add one or more conv layers
        conv_blocks = []

        # ability to  add one or more conv blocks
        for _ in range(n_conv_blocks):
            conv_blocks += [
                SAGEConv(n_node_features, n_hidden),
                Tanh(),
                SAGEConv(n_hidden, n_hidden),
                Tanh(),
            ]

        # group all the conv layers
        self.conv_layers = ModuleList(conv_blocks)

        ## Summary statistics
        self.summary_statistics = Sequential(
            Linear(n_hidden, n_hidden),
            Tanh(),
            Linear(n_hidden, 2))
        
        self.loss = torch.nn.CrossEntropyLoss()


    def forward(self,
                x: torch.Tensor,
                edge_index: torch.Tensor,
                batch: torch.Tensor) -> torch.Tensor:
        """ Processing the GNN model.
        Args:
            x (torch.Tensor): Input features at node level.
            edge_index (torch.Tensor): Index pairs of vertices
        Returns:
            torch.Tensor: output tensor.
        """
        for layer in self.conv_layers:
            if isinstance(layer, SAGEConv) or isinstance(layer, GCNConv):
                x = layer(x, edge_index)
            else:
                x = layer(x)
        x = gap(x, batch)
        pi   = self.summary_statistics(x)
        return F.softmax(pi,dim=1)
    


    def loss_func(self, out, y):
        out=out.to(DEVICE)
        loss_cross_entropy = self.loss(out, y)
        # dist=Categorical(out)
        # action=dist.sample().numpy()[0] #take a sample from the categorical dist from 1-22
        # print(action)
        # loss_extreme= torch.mean(-1*torch.log(action))
        
        # sample = (self.dist.log_prob(self.dist.sample_n(32).squeeze(1)).exp() ).to(DEVICE)
        # out=torch.sigmoid(out).to(DEVICE)
        # loss_extreme = torch.mean(out-sample- torch.mul(out,\
        #                torch.log( out/(sample+1e-10) ) ) )
        return (loss_cross_entropy)


def train(model, loader):
    """ Train function

    Args:
        model (object): GNN model instance.
        loader (pyg.loader.DataLoader): Data loader object.

    Returns:
        float: Training accuracy.
    """
    model.train()
    total_loss = 0
    out_ret = []
    for data in loader:
        data = data.to(DEVICE)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = model.loss_func(out.float(), data.y)
        loss.backward()
        optimizer.step()
        total_loss += float(loss.cpu().detach()) * data.num_graphs
        out_ret+= out.squeeze(1).cpu().detach().numpy().tolist()
    return total_loss / len(loader.dataset), out_ret, model



import sklearn
@torch.no_grad()
def test(model, loader):
    """ Evaluation function.

    Args:
        model (object): GNN model instance.
        loader (pyg.loader.DataLoader): Data loader object.

    Returns:
        tuple (float, list): Testing accuracy, predicted labels.
    """
    model.eval()
    total_correct = 0
    y_pred = []
    outs = []
    y_true = []
    for data in loader:
        data = data.to(DEVICE)
        _, pred = torch.max(model(data.x, data.edge_index, data.batch), 1)
        y_pred += pred.detach().cpu().numpy().tolist()
        y_true += data.y.detach().cpu().numpy().tolist()
    accuracy = sklearn.metrics.accuracy_score(y_pred, y_true)
    return accuracy, y_pred, y_true, outs

## The upper limits for each workflow

In [23]:
workflows = ["all", "1000genome", 
        "nowcast-clustering-8",
        "nowcast-clustering-16",
        "wind-clustering-casa",
        "wind-noclustering-casa"]
tasks = []
task_loader = []
for wf in workflows:
    if wf == "all":
        dataset = Merge_PSD_Dataset(node_level=False, binary_labels=True, anomaly_level=None).shuffle()
    else:
        dataset = PSD_Dataset("./", wf,
                              node_level=False,
                              binary_labels=True,
                              anomaly_level=None ).shuffle()
        
    temp_dataset = dataset
    n_graphs = len(temp_dataset)
    train_idx, val_idx = train_test_split( np.arange(n_graphs), train_size=0.6, random_state=0, shuffle=True)
    # print(train_idx, test_idx, val_idx)
    train_loader = DataLoader(temp_dataset[train_idx], batch_size=32)
    val_loader = DataLoader(temp_dataset[val_idx], batch_size=32)
    test_loader = DataLoader(dataset, batch_size=32)
    task_loader.append( (train_loader, val_loader, test_loader))

NUM_NODE_FEATURES = dataset.num_node_features
NUM_OUT_FEATURES = dataset.num_classes
print(NUM_NODE_FEATURES, NUM_OUT_FEATURES)

''' Build GNN model '''
from torch.utils.tensorboard import SummaryWriter
for (loaders, wfs) in zip(task_loader, workflows):
    # default `log_dir` is "runs" - we'll be more specific here
    writer = SummaryWriter('runs/'+wfs)

    model = GNN(NUM_NODE_FEATURES,64, NUM_OUT_FEATURES).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-04)
    scheduler = ReduceLROnPlateau(optimizer, 'min')

    (train_loader, val_loader, test_loader) = loaders
    ts_start = datetime.now().strftime('%Y%m%d_%H%M%S')
    pbar = tqdm(range(1000), desc=wfs)
    best = 0
    loss_array = []

    for e in pbar:
        model.train()
        optimizer.zero_grad()
        train_loss, outputs, model = train(model, train_loader)
        train_acc, y_pred, _, _    = test(model, train_loader)
        val_acc, _, _, _           = test(model, val_loader)
        pbar.set_postfix({"train_loss": train_loss, "train_acc": train_acc, "val_acc": val_acc})
        scheduler.step(train_loss)

        loss_array.append(train_loss)
        # ...log the running loss
        writer.add_scalar('training loss', train_loss, e)
        writer.add_scalar('training accuracy', train_acc, e)
        writer.add_scalar('Validation accuracy', val_acc, e)

        if e % 10 == 0:
            # ## The resulting information
            # actual_Data = model.dist.sample_n( 10000 )
            # probs = model.dist.log_prob(actual_Data).exp().squeeze(1)
            # args = np.argsort(actual_Data.squeeze(1).detach().cpu().numpy() )
            # probs = probs[args]
            # actual_Data = actual_Data[args]
            # train_acc, y_pred, y_true, outputs = test(model, train_loader)
            # plt.plot(actual_Data, probs)
            # x, bins, p = plt.hist(outputs, density=True)
            # for item in p:
            #     item.set_height(item.get_height()/sum(x))
            # # plt.xlim([0,1])
            # # plt.ylim([0,1])
            # plt.title(wfs)
            # # plt.savefig('CL_Figures/'+wfs+'prob'+'.png', dpi = 500)
            # plt.show()
            # ## The final training metrics
            # ys = []    
            test_acc, y_pred, y_true, _ = test(model, test_loader)
            print(np.unique(y_pred, return_counts=True), "real", np.unique(y_true, return_counts=True) )
            conf_mat = confusion_matrix(y_true, y_pred)
            prec_val = precision_score(y_true, y_pred)
            f1_val = f1_score(y_true, y_pred)
            recall_val = recall_score(y_true, y_pred)
            print("graph level clf:", wfs,
                f"binary True",
                f"test acc {test_acc:.4f}",
                f"f1 {f1_val:.4f}",
                f"recall {recall_val:.4f}",
                f"prec {prec_val:.4f}", conf_mat, "\n"
                )
    np.savetxt('CL_res/'+wfs+'upper_training.csv', np.array(loss_array), delimiter = ',')
    break

24 2


all:   1%|          | 1/100 [00:01<01:54,  1.16s/it, train_loss=0.609, train_acc=0.792, val_acc=0.796]

(array([1]), array([6149])) real (array([0, 1]), array([1270, 4879]))
graph level clf: all binary True test acc 0.7935 f1 0.8848 recall 1.0000 prec 0.7935 [[   0 1270]
 [   0 4879]] 



all:  11%|█         | 11/100 [00:06<00:55,  1.60it/s, train_loss=0.522, train_acc=0.792, val_acc=0.796]

(array([1]), array([6149])) real (array([0, 1]), array([1270, 4879]))
graph level clf: all binary True test acc 0.7935 f1 0.8848 recall 1.0000 prec 0.7935 [[   0 1270]
 [   0 4879]] 



...

all:  81%|████████  | 81/100 [00:47<00:11,  1.59it/s, train_loss=0.522, train_acc=0.792, val_acc=0.796]

(array([1]), array([6149])) real (array([0, 1]), array([1270, 4879]))
graph level clf: all binary True test acc 0.7935 f1 0.8848 recall 1.0000 prec 0.7935 [[   0 1270]
 [   0 4879]] 



all:  91%|█████████ | 91/100 [00:52<00:05,  1.59it/s, train_loss=0.522, train_acc=0.792, val_acc=0.796]

(array([1]), array([6149])) real (array([0, 1]), array([1270, 4879]))
graph level clf: all binary True test acc 0.7935 f1 0.8848 recall 1.0000 prec 0.7935 [[   0 1270]
 [   0 4879]] 



all: 100%|██████████| 100/100 [00:57<00:00,  1.73it/s, train_loss=0.522, train_acc=0.792, val_acc=0.796]


In [5]:
workflows = ["1000genome", "nowcast-clustering-8",
        "nowcast-clustering-16",
        "wind-clustering-casa",
        "wind-noclustering-casa"]

tt = Merge_PSD_Dataset(node_level=False, binary_labels=True, anomaly_level=None).shuffle()
test_loader = DataLoader(tt, batch_size=32)
tasks = []
for wf in workflows:
    dataset = PSD_Dataset("./", wf,
                            node_level=False,
                            binary_labels=True,
                            anomaly_level=None ).shuffle()
    tasks.append(dataset)

NUM_NODE_FEATURES = dataset.num_node_features
NUM_OUT_FEATURES  = dataset.num_classes
print(NUM_NODE_FEATURES, NUM_OUT_FEATURES)

''' Build GNN model '''
model = GNN(NUM_NODE_FEATURES, 64, NUM_OUT_FEATURES).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-03)
scheduler = ReduceLROnPlateau(optimizer, 'min')

24 2


## The naive training for each workflow.

In [6]:
from torch.utils.tensorboard import SummaryWriter
acc_arr = []
f1_arr = []
precision_arr = []
recall_arr = [] 
loss_array = []
for (wfs, dataset) in zip(workflows, tasks):
    print("The one I am learning right now", wfs)
    n_graphs = len(dataset)
    train_idx, test_idx = train_test_split( np.arange(n_graphs),\
                        train_size=0.6,\
                        random_state=0,\
                        shuffle=True)


    train_loader = DataLoader(dataset[train_idx], batch_size=32)
    writer = SummaryWriter('runs/'+wfs)
    ts_start = datetime.now().strftime('%Y%m%d_%H%M%S')
    pbar = tqdm(range(1000), desc=wfs)
    best = 0
    for e in pbar:
        train_loss, outputs, model   = train(model, train_loader)
        train_acc, y_pred, _, _= test(model, train_loader)
        test_acc, y_pred, y_true, _ = test(model, test_loader)

        conf_mat = confusion_matrix(y_true, y_pred)
        prec_val = precision_score(y_true, y_pred)
        f1_val = f1_score(y_true, y_pred)
        recall_val = recall_score(y_true, y_pred)
        
        pbar.set_postfix({"train_loss": train_loss,\
                        "train_acc": train_acc,\
                        "test_acc": test_acc,\
                        "f1 score": f1_val})
        
        scheduler.step(train_loss)
        
        # ...log the running loss
        writer.add_scalar('training loss', train_loss, e)
        writer.add_scalar('training accuracy', train_acc, e)
        writer.add_scalar('Test accuracy', test_acc, e)


        loss_array.append(train_loss)
        acc_arr.append(test_acc)
        f1_arr.append(f1_val)
        precision_arr.append(prec_val)
        recall_arr.append(recall_val)
        if e %100 == 0:
            print("graph level clf:",
                wfs,
                f"binary True",
                f"test acc {test_acc:.4f}",
                f"f1 {f1_val:.4f}",
                f"recall {recall_val:.4f}",
                f"prec {prec_val:.4f}", conf_mat, "\n"
                )

np.savetxt('CL_res/Naive_training_loss.csv', np.array(loss_array), delimiter = ',')
np.savetxt('CL_res/Naive_acc.csv', np.array(acc_arr), delimiter = ',')
np.savetxt('CL_res/Naive_precision.csv', np.array(precision_arr), delimiter = ',')
np.savetxt('CL_res/Naive_recall.csv', np.array(recall_arr), delimiter = ',')

The one I am learning right now 1000genome


1000genome:   0%|          | 1/1000 [00:23<6:25:26, 23.15s/it, train_loss=0.346, train_acc=0.854, test_acc=0.793, f1 score=0.885]

graph level clf: 1000genome binary True test acc 0.7935 f1 0.8848 recall 1.0000 prec 0.7935 [[   0 1270]
 [   0 4879]] 



1000genome:  10%|█         | 101/1000 [02:57<22:57,  1.53s/it, train_loss=0.0274, train_acc=0.903, test_acc=0.804, f1 score=0.889]

graph level clf: 1000genome binary True test acc 0.8037 f1 0.8886 recall 0.9869 prec 0.8082 [[ 127 1143]
 [  64 4815]] 



...

wind-noclustering-casa:  80%|████████  | 801/1000 [18:53<04:40,  1.41s/it, train_loss=1.37, train_acc=0.773, test_acc=0.804, f1 score=0.889]

graph level clf: wind-noclustering-casa binary True test acc 0.8044 f1 0.8888 recall 0.9854 prec 0.8094 [[ 138 1132]
 [  71 4808]] 



wind-noclustering-casa:  90%|█████████ | 901/1000 [21:14<02:19,  1.41s/it, train_loss=1.39, train_acc=0.773, test_acc=0.804, f1 score=0.889]

graph level clf: wind-noclustering-casa binary True test acc 0.8044 f1 0.8888 recall 0.9854 prec 0.8094 [[ 138 1132]
 [  71 4808]] 



wind-noclustering-casa: 100%|██████████| 1000/1000 [23:34<00:00,  1.41s/it, train_loss=1.42, train_acc=0.773, test_acc=0.804, f1 score=0.889]


The CL function here